In [9]:
import os
import cv2
import sys
import time
import numpy as np
from itertools import cycle
from threading import Thread, Event
# import simplejpeg

from multiprocessing import Queue
import copy
import imutils

 
class webCamVideoStream:
    def __init__(self, video_addr, readin_interval=3, cmp_interval=5, retry=10, maxqsize=30, im_w=None, name = "webCamera"):
        # 视频流地址
        self.video_addr = video_addr
        # 视频流
        self.stream = cv2.VideoCapture(self.video_addr)
        
        # 掉线重连次数
        self.retry = retry
        
        # 初始化摄像头
        self.stream_init()
        
        # 读帧时的跳帧设置
        self.readin_cycle_loop = cycle(list(range(readin_interval)))
        
        # 进程名
        self.name = name
        
        # 初始录制状态
        self.stopped = False
        
        # 图像队列
        self.imagedeque = Queue(maxsize=maxqsize) #deque(maxlen=maxqlen)
        
        self.current_qsize = 0
        
        # 图像宽度 （保持长宽比）
        self.im_w = im_w
        
        
        # 
        self.cur_frame = None
        self.prev_frame = None
        
        # 读帧时的跳帧设置
        self.cmp_cycle_loop = cycle(list(range(cmp_interval)))
        
        # 前后帧是否存在运动差
        self.no_motion = False
        
        

        
    def start(self):
        t = Thread(target=self.update, name=self.name, args=())
        t.daemon = True
        t.start()
        return self
    
    def update(self):
        while True:
            # 如果接收到停止命令，则停止读帧
            if self.stopped: return 
            
            self.grabbed, self.frame = self.stream.read()
            
            # 如果没有成功读帧，则重新初始化
            if not self.grabbed: self.stream_init()      
            
            # 执行跳帧
            self.skip_frame()
            
            # 执行rescale
            if self.im_w != None: self.frame = imutils.resize(self.frame, width = self.im_w)
                
#             # 对比相隔 cmp_interval的帧（跳帧后的帧顺序）。
            self.frame_cmp()
            
            self.imagedeque.put(self.frame)
            
            
                        
    def skip_frame(self):
        while True:
            if next(self.readin_cycle_loop) != 0:
                self.stream.grab()
            else:
                break
            
    def stream_init(self):
        self.grabbed, self.frame = self.stream.read()
        retry = self.retry
        
        if not self.grabbed:
            print(f"[INFO] webcam reconnectinig ")
            while retry:
                self.stream = cv2.VideoCapture(self.video_addr)
                self.grabbed, self.frame = self.stream.read()
                if not self.grabbed:
                    time.sleep(1)
                    print(retry)
                    retry -= 1
                    if retry == 0:
                        print(f"[INFO] webcam init failed !")
                        sys.exit(0)
                else:
                    break
                         
            
    def stop(self):
        self.stopped = True
        
        
    def read(self):
        return self.imagedeque.get()

    def frame_cmp(self):
        if next(self.cmp_cycle_loop) == 0:
            self.cur_frame = self.frame.copy()
            if isinstance(self.prev_frame, np.ndarray):
                score = self.frame_simi(self.cur_frame[:,:,0], self.prev_frame[:,:,0])
                if score <= 0.95:
                    self.no_motion = False
                else:
                    self.no_motion = True
                
                
            self.prev_frame = self.cur_frame
     
            
    def frame_simi(self, img1, img2, hash_type='phash'):
            
        def hash_score(hash1, hash2):
            hash_dist = sum([ham_dist(v1, v2) for v1, v2 in zip(hash1[0], hash2[0])])
            score = 1 - hash_dist / 64
            return score
        
        def ham_dist(x, y):
            """
            :type x: int
            :type y: int
            :rtype: int
            """
            return bin(x ^ y).count('1')
        
        try:
            h1, h2 = map(cv2.img_hash.pHash, (img1, img2,))
#             h1, h2 = map(cv2.img_hash.averageHash, (img1, img2,))
            score = hash_score(h1, h2)
            return score
        except Exception as e:
            print(e)
            return 0
        